# Interactive exploration of networks

This notebook demonstrates the interactive exploration capabilities of PyPSA networks.

In [ ]:
import pypsa

In [ ]:
# Load the SciGRID-DE example network
n = pypsa.examples.scigrid_de()

Lets optimize the SciGRID-DE example network to obtain flows and dispatches.

In [ ]:
n.optimize()

Using `n.statistics()` we can extract key results. Note that links do not exist in this network. By dividing through the number of snapshots, we get mean values.

In [ ]:
bus_sizes = n.statistics.supply(
    groupby=["bus"],
    comps=["Generator", "StorageUnit"],
    aggregate_across_components=True,
).div(len(n.snapshots))
line_widths = n.lines_t.p0.sum(axis=0).div(len(n.snapshots))
transformer_widths = n.transformers_t.p0.sum(axis=0).div(len(n.snapshots))

We use scaling factors to adjust to our needs.

In [ ]:
bus_sizes_factor = 10
line_factor = 5
arrow_size_factor = 2

# Interactive plot

In [ ]:
map = n.explore(
    bus_sizes=bus_sizes * bus_sizes_factor,
    bus_colors="deeppink",
    line_flow=line_widths * line_factor,
    line_colors="orange",
    line_widths=line_widths * line_factor,
    line_columns=["s_nom_opt", "voltage"],
    transformer_widths=transformer_widths * line_factor,
    arrow_colors="orange",
    arrow_size_factor=arrow_size_factor,
    map_style="dark",
)

In [ ]:
map

# Multi-index bus sizes, pie charts

To explore multi-index bus sizes, e.g. supply colored by carriers mapped to individual buses, we need to obtain the results via `n.statistics()`

In [ ]:
supply = n.statistics.supply(
    groupby=["bus", "carrier"], aggregate_across_components=True, comps=["Generator"]
)

We also need to set colors for all carriers.

In [ ]:
n.carriers.loc["Solar", "color"] = "yellow"
n.carriers.loc["Wind Onshore", "color"] = "blue"
n.carriers.loc["Run of River", "color"] = "darkblue"
n.carriers.loc["Waste", "color"] = "brown"
n.carriers.loc["Brown Coal", "color"] = "brown"
n.carriers.loc["Storage Hydro", "color"] = "cyan"
n.carriers.loc["Other", "color"] = "grey"
n.carriers.loc["Hard Coal", "color"] = "black"
n.carriers.loc["Nuclear", "color"] = "darkred"
n.carriers.loc["Wind Offshore", "color"] = "green"
n.carriers.loc["Gas", "color"] = "orange"
n.carriers.loc["Geothermal", "color"] = "purple"

Let's pass the supply results to the function method used previously.

In [ ]:
map = n.explore(
    bus_sizes=supply,
    bus_colors="deeppink",
    line_flow=line_widths * line_factor,
    line_colors="orange",
    line_widths=line_widths * line_factor,
    line_columns=["s_nom_opt", "voltage"],
    transformer_widths=transformer_widths * line_factor,
    arrow_colors="orange",
    arrow_size_factor=arrow_size_factor,
    map_style="dark",
)

To export to a self-contained html, run:

In [ ]:
map.to_html("interactive_map.html", offline=True)